Clase 9
===

* Encontrar la solución más semejante a datos experimentales


Medir calidad de las simulaciones
--------------------------------------

Si conocemos el valor de uno o varios flujos metabólicos podemos ajustar nuestro modelo a estos datos. Hay varias maneras de hacer esto, la primera es simplemente fijar el valor de los flujos como veremos a continuación. 

A modo de ejemplo usaremos Saccharomyces cerevisae y los flujos experimentales compilados por [García et al 2012](https://doi.org/10.1371/journal.pone.0043006)


In [56]:
# Let's import the yeast model
import cobra
model = cobra.io.read_sbml_model("iMM904.xml")

In [57]:
# And check the solution using default parameters
solution=model.optimize()
model.summary()

IN FLUXES          OUT FLUXES        OBJECTIVES
-----------------  ----------------  ----------------------
glc__D_e  10       co2_e   18        BIOMASS_SC5_...  0.288
o2_e       2       etoh_e  15.8
nh4_e      1.61    h2o_e    5.64
pi_e       0.0569  h_e      1.45
so4_e      0.0223  for_e    0.00149


In [93]:
# Now, let's modify the ethanol flux
# First, lets find the reactions associated with ethanol
pyr_suma_neta=0
for reaction in model.metabolites.get_by_id("pyr_c").reactions:
    print reaction, reaction.flux
    pyr_suma_neta=pyr_suma_neta+reaction.flux
print pyr_suma_neta

D_LACt2m: lac__D_c + pyr_m <=> lac__D_m + pyr_c 0.0
ALATA_L: akg_c + ala__L_c <=> glu__L_c + pyr_c -0.062080088372
THZPSN1_SC: achms_c + cys__L_c + gly_c + h_c + xu5p__D_c --> 4abut_c + 4mpetz_c + ac_c + co2_c + 3.0 h2o_c + nh4_c + pyr_c 0.0
CYSTL: cyst__L_c + h2o_c --> hcys__L_c + nh4_c + pyr_c 0.0
PYRDC2: acald_c + h_c + pyr_c --> actn__R_c + co2_c 0.0208736773286
PC: atp_c + hco3_c + pyr_c --> adp_c + h_c + oaa_c + pi_c 0.238570483078
ANS: chor_c + gln__L_c --> anth_c + glu__L_c + h_c + pyr_c 0.00345085144108
CHRPL: chor_c --> 4hbz_c + pyr_c 0.0
AGTi: ala__L_c + glx_c --> gly_c + pyr_c 0.00633182635897
L_LACD2cm: 2.0 ficytc_m + lac__L_c --> 2.0 focytc_m + pyr_c 0.0
PYRt2m: h_c + pyr_c <=> h_m + pyr_m 0.862253392951
PYRt2: h_e + pyr_e --> h_c + pyr_c 0.0
SERD_L: ser__L_c --> nh4_c + pyr_c 0.0
THZPSN2_SC: achms_c + cys__L_c + gly_c + h_c + r5p_c --> 4abut_c + 4mpetz_c + ac_c + co2_c + 3.0 h2o_c + nh4_c + pyr_c 0.0
PYRt2p: h_c + pyr_c <=> h_x + pyr_x 0.0
D_LACDcm: 2.0 ficytc_m + lac__D

In [99]:
# Fix the ethanol reaction flux and see how it affects the solution

model.reactions.get_by_id("EX_glc__D_e").upper_bound=-3.002
model.reactions.get_by_id("EX_glc__D_e").lower_bound=-3.002


solution=model.optimize()
model.summary()

IN FLUXES          OUT FLUXES        OBJECTIVES
-----------------  ----------------  ----------------------
glc__D_e  3        co2_e   5.61      BIOMASS_SC5_...  0.119
o2_e      2        etoh_e  3.63
nh4_e     0.875    h2o_e   3.61
pi_e      0.0235   h_e     1.03
so4_e     0.00917  ac_e    0.224
                   gly_e   0.211
                   for_e   0.000614


Es posible que errores de medición resulten en datos de flujos metabólicos que no son estequiometricamente compatibles. Para estos casos, una alternativa es fijar un rango de valores en lugar de un valor exacto. Por ejemplo:

In [54]:
# Fix a range for the ethanol reaction flux around +/-10% of the observed value
value=3.632
model.reactions.get_by_id("EX_etoh_e").upper_bound=value+0.1*value 
model.reactions.get_by_id("EX_etoh_e").lower_bound=value-0.1*value

solution=model.optimize()
model.summary()

IN FLUXES         OUT FLUXES        OBJECTIVES
----------------  ----------------  ----------------------
glc__D_e  3       co2_e   5.94      BIOMASS_SC5_...  0.122
o2_e      2       etoh_e  3.85
nh4_e     0.681   h2o_e   3.54
pi_e      0.024   h_e     0.612
so4_e     0.0094  for_e   0.000629


Ejercicio
---------

Usando los valores experimentales compilados por [García et al 2012](https://doi.org/10.1371/journal.pone.0043006) desde la referencia [20] (ver la columna **References** en Table_S1.doc):

1. Fija la reacción de consumo de glucosa (hay un pequeño problema que espero puedas descubrir por ti mismo).
2. Fija el resto de las reacciones a datos experimentales (haciendo 1 esto debería ser trivial).

In [132]:
import numpy as np
def normaE(x,y):
    sum=0
    for i in range(len(x)):
        sum=sum+np.power(x[i]-y[i],2)
    return sum
x=[0,1,3]
y=[0,2,5]
normaE(x,y)
np.arange(0,len(x))

array([0, 1, 2])

Evaluación de la calidad de las predicciones
------------------

Para evaluar que tan cercana es la simulación de FBA a los datos experimentales podemos usar la norma Euclideana la cual mide la distancia entre dos puntos en el hiperespacio:
    
\begin{align}
    \mbox{Norma Euclideana}=\sum_i (x_i-y_i)^2
\end{align}

La cual puede ser escrita usando notación vectorial:

\begin{align}
    \mbox{Norma Euclideana} = (x-y)^T(x-y)
\end{align}


Ejercicio
-----------

Escribe una función para determinar la distancia entre los datos simulados y los experimentales (usa los datos de la Ref 20) usando la norma Euclideana como métrica.


Optimización en dos niveles
-------------------------------

Cuando un FBA tiene multiples soluciones es de interes encontrar aquella que más se asemeje a datos experimentales. Solo en ese caso sería justo evaluar la capacidad de predicción del modelo. Esto se puede hacer formulando una problema de optimización en dos niveles. Mientras que en el nivel más interno se busca maximizar biomasa (esta es la formulación regular de FBA), en el nivel más externo se minimiza la distancia entre flujos simulados y los experimentales.

\begin{align}
\mbox{min}\ & \sum_{i \in I} ||x_i-y_i|| \\
\mbox{s.a.} & \\
&\mbox{max}\  f(x)=c^Tx \\
&\ \mbox{s.a.}  \\
&\ Sx=0 \\
&\ LB<=x<=UB
\end{align}

En donde $\sum_{i \in I} ||x_i-y_i||$ corresponde a una métrica arbitraria, por ejemplo la norma Euclideana (otras opciones puede ser la suma de las diferencias en valores absolutos).

¿Cómo se puede resolver un problema de optimización que depende de otro problema de optimización? Respuesta, usando la [teoría de dualidad](https://en.wikipedia.org/wiki/Linear_programming#Duality) de la programación lineal. En terminos concretos, si tenemos un problema en formato estandar que llamaremos 'primal':

\begin{align}
\mbox{min}\  & c^Tx \\
\mbox{s.a.}&  \\
&\ Ax=b \\
&\ x \ge 0
\end{align}

entonces existe un problema 'dual':

\begin{align}
\mbox{max}\  & b^Ty \\
\mbox{s.a.} & \\
&\ A^Ty \le c
\end{align}

Si existe una solución del primero entonces existe una solución en el segundo y el valor de la función objetivo es el mismo. Esto es útil en un problema de optimización anidado ya que podemos transformar el problema de optimización interno en el siguiente conjunto de restricciones:

\begin{align}
& c^Tx = b^Ty \\
&\ Ax=b \\
&\ A^Ty \le c \\
&\ x \ge 0
\end{align}

Para aplicar esta técnica a FBA primero debemos transformar la formulación de FBA en el formato estandar, lo cual veremos en el pizarron como hacerlo.

Tarea
-------

Estudia el uso de esta técnica en el paper de [Burgard et al 2003](http://onlinelibrary.wiley.com/doi/10.1002/bit.10803/epdf).

Ejercicio
-----------

Dada la (mini) red metabólica mostrada en la figura de abajo:

<img src="https://raw.githubusercontent.com/modcommet/Clases/master/miniRed_clase8.png" alt="Drawing" style="width: 200px;"/>

* Formular el FBA maximizando $x_4$.
* Representar el FBA en forma estandar de un problema de programación lineal, el cual corresponde al primal.
* Construir el problema dual.
* Automatizar este proceso construyendo una algoritmo que tome como inputs: $c$, $S$, $x_{UB}$, y $x_{LB}$. 

Continumeos desde los resultados de la clase anterior:

![Image](https://raw.githubusercontent.com/modcommet/Clases/master/pizarra_clase8.jpg)

Resumen del problema original (minimizar norma Euclideana)
------------------------------------------------------------------

Usando la técnica anterior hemos reducido nuestro problema de dos optimizaciones anidadas a una optimización simple:

\begin{align}
\mbox{min}\ & ||M\bar{x}-\bar{x}^o||^2 \\
\mbox{s.a.} & \\
& \bar{c}^T\bar{x} -b^Ty =0 \\
&\ \bar{A}\bar{x}=b \\
&\ \bar{A}^Ty \le \bar{c} \\
&\ \bar{x} \ge 0
\end{align}

¿Cómo se construyen todas esas nuevas matrices y vectores? Que el computador haga el trabajo por nosotros

In [18]:
import numpy as np

def hacerPrimalDual(A,c,x_l,x_u):
    # get number of (M)etabolites and (R)eactions
    M,R = A.shape
    # define empty matrices with the appropiate dimensions
    c_bar = np.zeros((2*R, 1 ))
    A_bar = np.zeros((M+R,2*R))
    b = np.zeros((M+R, 1 ))
    # fill matrices
    c_bar[:R] = -c
    A_bar[:M,:R] = S
    A_bar[M:,:R] = np.identity(R)
    A_bar[M:,R: ] = np.identity(R)
    b[:M] = -A*x_l
    b[M:] = x_u-x_l
    return c_bar,A_bar,b

c=np.matrix([0,0,0,1]).reshape((4,1))
S=np.matrix([[1,-1,-1,0],[0,1,1,-1]])
x_l=np.matrix([-1000]*4).reshape((4,1))
x_u=np.matrix([10]+[1000]*3).reshape((4,1))
c_bar,A_bar,b = hacerPrimalDual(S,c,x_l,x_u)

Resolución en python
--------------------------------------

Un problema de optimización cuadrática es su forma canónica es:

\begin{align}
\mbox{min}\ & 1/2\bar{\bar{x}}^TP\bar{\bar{x}}-q\bar{\bar{x}} \\
\mbox{s.a.} & \\
&\ \bar{\bar{A}}\bar{\bar{x}}=\bar{b} \\
& G\bar{\bar{x}} \le h
\end{align}

En donde la función objetivo resulta de expander el cuadrado de la norma Euclideana:

\begin{align}
\mbox{min}\ ||M\bar{\bar{x}}-\bar{\bar{x}}^o||^2 &= \mbox{min}\ 1/2\bar{\bar{x}}^T(M^TM)\bar{\bar{x}}-(M^T\bar{\bar{x}}^o)^T\bar{\bar{x}} \\
\mbox{En donde:} \\
&P=(M^TM) \\
&q=(M^T\bar{\bar{x}}^o)^T
\end{align}


Para resolver este tipo de problemas podemos instalar [CVXOPT](http://cvxopt.org/userguide/intro.html), un paquete para minimizar funciones objetivos cuadráticas (como es el caso de la norma Euclideana) sujeto a restricciones de igualdad y desigualdad. Esto se puede hacer abriendo el terminal de anaconda e ingresando el siguiente comando: `pip install cvxopt`.

Ejercicio
-----------

Resolver el siguiente problema (original [aquí](https://scaron.info/blog/quadratic-programming-in-python.html)) con cvxopt:

<img src="https://raw.githubusercontent.com/modcommet/Clases/master/quadratic_example_clase9.png" alt="Drawing" style="width: 400px;"/>

In [19]:
# Load libraries
import cvxopt
from cvxopt import matrix
import numpy as np
# Defines a wraper for cvxopt 
# Among other things it transforms matrices from numpy to cvxopt format
def cvxopt_solve_qp(P, q, G=None, h=None, A=None, b=None):
    P = .5 * (P + P.T)  # make sure P is symmetric
    args = [matrix(P), matrix(q)]
    if G is not None:
        args.extend([matrix(G), matrix(h)])
        if A is not None:
            args.extend([matrix(A), matrix(b)])
    sol = cvxopt.solvers.qp(*args)
    if 'optimal' not in sol['status']:
        return None
    return np.array(sol['x']).reshape((P.shape[1],))
# Define matrices    
M = np.array([[1., 2., 0.], [-8., 3., 2.], [0., 1., 1.]])
P = np.dot(M.T, M)
q = np.dot(np.array([3., 2., 3.]), M).reshape((3,))
G = np.array([[1., 2., 1.], [2., 0., 1.], [-1., 2., -1.]])
h = np.array([3., 2., -2.]).reshape((3,))
# The results:
cvxopt_solve_qp(P, q, G, h)

     pcost       dcost       gap    pres   dres
 0: -3.2822e+00 -9.6005e+00  6e+00  5e-17  9e-01
 1: -6.9907e+00 -8.3734e+00  1e+00  3e-16  3e-01
 2: -1.0434e+01 -1.1566e+01  1e+00  2e-16  5e-16
 3: -1.0492e+01 -1.0506e+01  1e-02  2e-16  5e-16
 4: -1.0495e+01 -1.0495e+01  1e-04  2e-16  3e-16
 5: -1.0495e+01 -1.0495e+01  1e-06  3e-16  4e-16
Optimal solution found.


array([-0.49025721, -1.57755278, -0.66484775])

Tarea
---------

Resolver el problema de la red métabolica.

1. Escribir una función que tome como argumentos $c$, $A$, $x^u$, y $x^l$, entregue como resultado $G$, $h$, $\bar{\bar{A}}$, $\bar{b}$.
2. Asumiendo que los datos observados son $x_1=10$, y $x_2=3$, definir $M$,$x^o$, y $P$, y $q$.
3. Utilizar cvxopt para encontrar la solución.